In [2]:
import os
import time
from PIL import Image
from torchvision import transforms, models
from torch.utils.data import DataLoader, Dataset, random_split
import pytorch_lightning as pl
import torch
from sklearn.metrics import accuracy_score, f1_score
from tqdm import tqdm
from timm import create_model  # For DeiT

In [3]:
# Hyperparameters
BATCH_SIZE = 32
LEARNING_RATE = 2e-5
EPOCHS = 5
IMG_SIZE = 224  # Image size for DeiT
DATASET_DIR = '/kaggle/input/lra-pathfinder-32/pathfinder32/curv_contour_length_14'  # Update as needed

In [4]:
# Dataset Class
class PathfinderDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path, label = self.data[idx]
        try:
            image = Image.open(img_path).convert('RGB')
            if self.transform:
                image = self.transform(image)
            return image, label
        except (IOError, OSError, Image.DecompressionBombError, Image.UnidentifiedImageError):
            new_idx = (idx + 1) % len(self.data)
            return self[new_idx]

In [5]:
# Data Module
class PathfinderDataModule(pl.LightningDataModule):
    def __init__(self, dataset_dir, batch_size=BATCH_SIZE):
        super().__init__()
        self.dataset_dir = dataset_dir
        self.batch_size = batch_size
        self.transform = transforms.Compose([
            transforms.Resize((IMG_SIZE, IMG_SIZE)),
            transforms.ToTensor(),
        ])

    def prepare_data(self):
        self.data_list = []
        metadata_dir = os.path.join(self.dataset_dir, 'metadata')
        for file_name in os.listdir(metadata_dir):
            metadata_path = os.path.join(metadata_dir, file_name)
            with open(metadata_path, 'r') as file:
                for line in file:
                    tokens = line.strip().split()
                    img_rel_path = tokens[0] + "/" + tokens[1]
                    label = int(tokens[3])
                    img_path = os.path.join(self.dataset_dir, img_rel_path)
                    self.data_list.append((img_path, label))

    def setup(self, stage=None):
        dataset = PathfinderDataset(self.data_list, transform=self.transform)
        train_size = int(0.8 * len(dataset))
        val_size = int(0.1 * len(dataset))
        test_size = len(dataset) - train_size - val_size
        self.train_set, self.val_set, self.test_set = random_split(dataset, [train_size, val_size, test_size])

    def train_dataloader(self):
        return DataLoader(self.train_set, batch_size=self.batch_size, shuffle=True, num_workers=4)

    def val_dataloader(self):
        return DataLoader(self.val_set, batch_size=self.batch_size, num_workers=4)

    def test_dataloader(self):
        return DataLoader(self.test_set, batch_size=self.batch_size, num_workers=4)


In [6]:
# Initialize Data Module
data_module = PathfinderDataModule(DATASET_DIR)
data_module.prepare_data()
data_module.setup()

In [7]:
# Initialize DeiT model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = create_model('deit_small_patch16_224', pretrained=True, num_classes=2)
model.to(device)

model.safetensors:   0%|          | 0.00/88.2M [00:00<?, ?B/s]

VisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 384, kernel_size=(16, 16), stride=(16, 16))
    (norm): Identity()
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (patch_drop): Identity()
  (norm_pre): Identity()
  (blocks): Sequential(
    (0): Block(
      (norm1): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=384, out_features=1152, bias=True)
        (q_norm): Identity()
        (k_norm): Identity()
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=384, out_features=384, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (ls1): Identity()
      (drop_path1): Identity()
      (norm2): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=384, out_features=1536, bias=True)
        (act): GELU(approximate='none')
        (drop1): Dropout(p=0.0, inplace=False)
        (norm): Identity(

In [8]:
def train_model(model, train_loader, optimizer, loss_fn, epochs=EPOCHS):
    model.train()
    start_time = time.time()
    for epoch in range(epochs):
        print(f"Epoch {epoch+1}/{epochs}")
        total_loss = 0
        for images, labels in tqdm(train_loader, desc=f"Training Epoch {epoch+1}"):
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = loss_fn(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
        print(f"Epoch {epoch+1} Loss: {total_loss / len(train_loader):.4f}")
    end_time = time.time()
    return end_time - start_time

In [9]:
def evaluate_model(model, test_loader):
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for images, labels in tqdm(test_loader, desc="Evaluating"):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            preds = torch.argmax(outputs, dim=-1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds)
    return accuracy, f1

In [10]:
# Train and Evaluate
optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)
loss_fn = torch.nn.CrossEntropyLoss()

train_loader = data_module.train_dataloader()
test_loader = data_module.test_dataloader()

train_time = train_model(model, train_loader, optimizer, loss_fn, epochs=EPOCHS)
accuracy, f1 = evaluate_model(model, test_loader)

# Calculate number of parameters
num_params = sum(p.numel() for p in model.parameters())

# Calculate efficiency score
efficiency = accuracy / (torch.log(torch.tensor(train_time)) * torch.log(torch.tensor(num_params)))

Epoch 1/5


Training Epoch 1: 100%|██████████| 5000/5000 [33:25<00:00,  2.49it/s]


Epoch 1 Loss: 0.6940
Epoch 2/5


Training Epoch 2: 100%|██████████| 5000/5000 [33:31<00:00,  2.49it/s]


Epoch 2 Loss: 0.6935
Epoch 3/5


...

Training Epoch 4: 100%|██████████| 5000/5000 [33:36<00:00,  2.48it/s]


Epoch 4 Loss: 0.5638
Epoch 5/5


Training Epoch 5: 100%|██████████| 5000/5000 [33:39<00:00,  2.48it/s]


Epoch 5 Loss: 0.4368


Evaluating: 100%|██████████| 625/625 [01:26<00:00,  7.24it/s]


In [11]:
# Print metrics
print("\nTraining Metrics:")
print(f"Time Taken: {train_time:.2f} seconds")
print(f"Accuracy: {accuracy:.4f}")
print(f"Number of Parameters: {num_params}")
print(f"Efficiency Score: {efficiency.item():.4f}")

# Save the model
torch.save(model.state_dict(), 'deit_model.pth')


Training Metrics:
Time Taken: 10064.90 seconds
Accuracy: 0.7883
Number of Parameters: 21666434
Efficiency Score: 0.0051
